In [1]:
import pandas as pd
import glob

In [8]:
files = glob.glob('stimuli/*.png')

In [9]:
files

['stimuli\\Baracke.png',
 'stimuli\\Branflakes.png',
 'stimuli\\Branflakesextra.png',
 'stimuli\\Cafeelite.png',
 'stimuli\\Chocolatemilk.png',
 'stimuli\\ChocolateSpread.png',
 'stimuli\\CocaCola2L.png',
 'stimuli\\Colgate.png',
 'stimuli\\Filtuna.png',
 'stimuli\\Fitness.png',
 'stimuli\\Fitness_Chocolate.png',
 'stimuli\\FuzeTea.png',
 'stimuli\\FuzeTeaGreen.png',
 'stimuli\\GelDeLin.png',
 'stimuli\\HEINZ.PNG',
 'stimuli\\KifKef.PNG',
 'stimuli\\Kikkoman.PNG',
 'stimuli\\Milka.PNG',
 'stimuli\\Neviot.PNG',
 'stimuli\\Oreo.PNG',
 'stimuli\\Parachocolate.PNG',
 'stimuli\\Prince.PNG',
 'stimuli\\Quaker.PNG',
 'stimuli\\Sano99.PNG',
 'stimuli\\Smarties.PNG',
 'stimuli\\Splendid.PNG',
 'stimuli\\Tapuchips.PNG',
 'stimuli\\Tuna_ToGo.png',
 'stimuli\\Wissotzky.png',
 'stimuli\\YadMordechai.png',
 'stimuli\\Zipperbigbag.png',
 'stimuli\\Zippersmallbag.png']

In [10]:
stimuli_df = pd.DataFrame({'target1':files[:10], 'target2':files[10:20], 'distracter':files[20:30]})
stimuli_df.to_csv('stimuli.csv')

In [11]:
stimuli_df

,target1,target2,distracter
0,stimuli\Baracke.png,stimuli\Fitness_Chocolate.png,stimuli\Parachocolate.PNG
1,stimuli\Branflakes.png,stimuli\FuzeTea.png,stimuli\Prince.PNG
2,stimuli\Branflakesextra.png,stimuli\FuzeTeaGreen.png,stimuli\Quaker.PNG
3,stimuli\Cafeelite.png,stimuli\GelDeLin.png,stimuli\Sano99.PNG
4,stimuli\Chocolatemilk.png,stimuli\HEINZ.PNG,stimuli\Smarties.PNG
5,stimuli\ChocolateSpread.png,stimuli\KifKef.PNG,stimuli\Splendid.PNG
6,stimuli\CocaCola2L.png,stimuli\Kikkoman.PNG,stimuli\Tapuchips.PNG
7,stimuli\Colgate.png,stimuli\Milka.PNG,stimuli\Tuna_ToGo.png
8,stimuli\Filtuna.png,stimuli\Neviot.PNG,stimuli\Wissotzky.png
9,stimuli\Fitness.png,stimuli\Oreo.PNG,stimuli\YadMordechai.png


In [12]:
pd.DataFrame({'image':files}).to_csv('stimuli.csv')